Collect neuronal responses (dff and inferred spikes) from collected ROIs. <br>
Compare between those from master ROIs and from the session ROIs (check correlation distribution). <br>
Use lambda = 1. <br>
Use inverse transformations. For suite2p 2step nonrigid transformation, refer 220127_extract_signals_from_masterROI.py <br>
Continuation from 220627_roi_collection_and_QC.ipynb.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import napari
from suite2p.registration import rigid, nonrigid
from suite2p.gui.drawroi import masks_and_traces
from pystackreg import StackReg
import time
import os
import gc
gc.enable()

def stack_reg_reverse(img, tforms, reg_meth):
    '''
    Reverse StackReg registration
    Input:
        img: 2d or 3d images. or list of 2d images
        tforms: list of StackReg tmat 
        reg_meth: method of registration. Currently only 'affine' and 'bilinear' is allowed.
    Output:
        reverse_img_list: array of reverse-registered images.
    '''
    if isinstance(img,list):
        img = np.array(img)
    if len(img.shape)==2:
        img = np.expand_dims(img,axis=0)
    nimg = img.shape[0]
    assert nimg==len(tforms)
    if reg_meth == 'affine':
        sr = StackReg(StackReg.AFFINE)
    elif reg_meth == 'bilinear':
        sr = StackReg(StackReg.BILINEAR)
    else:
        raise('reg_meth input must be either affine or bilinear.')

    reverse_img_list = []
    for i in range(nimg):
        tmat = tforms[i]
        reverse_tmat = np.linalg.inv(tmat)
        reverse_img = sr.transform(img[i,:,:], tmat=reverse_tmat)
        reverse_img_list.append(reverse_img)
    return np.array(reverse_img_list)
        

def twostep_register_reverse(img, rigid_y1, rigid_x1, nonrigid_y1, nonrigid_x1, block_size1, 
                     rigid_y2, rigid_x2, nonrigid_y2, nonrigid_x2, block_size2):
    '''
    Reverse twostep_register
    All the inputs are the same as those used for the original twostep_register
    '''
    frames = img.copy().astype(np.float32)
    if len(frames.shape) == 2:
        frames = np.expand_dims(frames, axis=0)
    elif len(frames.shape) < 2:
        raise('Dimension of the frames should be at least 2')
    elif len(frames.shape) > 3:
        raise('Dimension of the frames should be at most 3')
    (Ly, Lx) = frames.shape[1:]
    
    # 1st nonrigid shift (reversing 2nd nonrigid)
    yblock, xblock, nblocks, block_size, NRsm = nonrigid.make_blocks(Ly=Ly, Lx=Lx, block_size=block_size2)
    ymax1 = np.tile(-nonrigid_y2, (frames.shape[0],1))
    xmax1 = np.tile(-nonrigid_x2, (frames.shape[0],1))
    frames = nonrigid.transform_data(data=frames, nblocks=nblocks, 
        xblock=xblock, yblock=yblock, ymax1=ymax1, xmax1=xmax1)
    
    # 1st rigid shift (reversing 2nd rigid)
    frames = np.roll(frames, (rigid_y2, rigid_x2), axis=(1,2))
    
    # 2nd nonrigid shift (reversing 1st nonrigid)
    yblock, xblock, nblocks, block_size, NRsm = nonrigid.make_blocks(Ly=Ly, Lx=Lx, block_size=block_size1)
    ymax1 = np.tile(-nonrigid_y1, (frames.shape[0],1))
    xmax1 = np.tile(-nonrigid_x1, (frames.shape[0],1))
    frames = nonrigid.transform_data(data=frames, nblocks=nblocks, 
        xblock=xblock, yblock=yblock, ymax1=ymax1, xmax1=xmax1)
    
    # 2nd rigid shift (reversing 1st rigid)
    frames = np.roll(frames, (rigid_y1, rigid_x1), axis=(1,2))
    
    return frames
    
def fix_reg_collection(reg_result):
    '''
    Fixing registration result collection of the 'new' suite2p method.
    2022/07/13 JK
    '''
    num_sessions = len(reg_result['selected_session_num'])
    roff1_y = [0]
    roff1_x = [0]
    roff1_c = [0.1]
    roff1 = reg_result['suite2p_result']['roff1']
    for i in range(num_sessions-1):
        roff1_y.append([x[0] for x in roff1[i]][0])
        roff1_x.append([x[0] for x in roff1[i]][1])
        roff1_c.append([x[0] for x in roff1[i]][2])
    roff1 = [[np.array(roff1_y), np.array(roff1_x), np.array(roff1_c)]]
    reg_result['suite2p_result']['roff1'] = roff1

    roff2_y = [0]
    roff2_x = [0]
    roff2_c = [0.1]
    roff2 = reg_result['suite2p_result']['roff2']
    for i in range(num_sessions-1):
        roff2_y.append([x[0] for x in roff2[i]][0])
        roff2_x.append([x[0] for x in roff2[i]][1])
        roff2_c.append([x[0] for x in roff2[i]][2])
    roff2 = [[np.array(roff2_y), np.array(roff2_x), np.array(roff2_c)]]
    reg_result['suite2p_result']['roff2'] = roff2

    offset_len = len(reg_result['suite2p_result']['nroff1'][0][0][0])
    nroff1_y = [np.zeros(offset_len)]
    nroff1_x = [np.zeros(offset_len)]
    nroff1_c = [np.ones(offset_len)/10]
    nroff1 = reg_result['suite2p_result']['nroff1']
    for i in range(num_sessions-1):
        nroff1_y.append([x[0] for x in nroff1[i]][0])
        nroff1_x.append([x[0] for x in nroff1[i]][1])
        nroff1_c.append([x[0] for x in nroff1[i]][2])
    nroff1 = [[np.array(nroff1_y).astype(np.float32), np.array(nroff1_x).astype(np.float32), np.array(nroff1_c).astype(np.float32)]]
    reg_result['suite2p_result']['nroff1'] = nroff1
    
    offset_len = len(reg_result['suite2p_result']['nroff2'][0][0][0])
    nroff2_y = [np.zeros(offset_len)]
    nroff2_x = [np.zeros(offset_len)]
    nroff2_c = [np.ones(offset_len)/10]
    nroff2 = reg_result['suite2p_result']['nroff2']
    for i in range(num_sessions-1):
        nroff2_y.append([x[0] for x in nroff2[i]][0])
        nroff2_x.append([x[0] for x in nroff2[i]][1])
        nroff2_c.append([x[0] for x in nroff2[i]][2])
    nroff2 = [[np.array(nroff2_y).astype(np.float32), np.array(nroff2_x).astype(np.float32), np.array(nroff2_c).astype(np.float32)]]
    reg_result['suite2p_result']['nroff2'] = nroff2

    return reg_result

def calculate_regCell_threshold(cellMap, numPix, thresholdResolution = 0.01):
    trPrecision = len(str(thresholdResolution).split('.')[1])
    thresholdRange = np.around(np.arange(0.3,1+thresholdResolution/10,thresholdResolution), trPrecision)
    threshold = thresholdRange[np.argmin([np.abs(numPix - np.sum(cellMap>=threshold)) for threshold in thresholdRange])]
    cutMap = (cellMap >= threshold).astype(bool)
    return cutMap, threshold

def imblend_for_napari(refImg, testImg):
    if (len(refImg.shape) != 2) or (len(testImg.shape) != 2):
        raise('Both images should have 2 dims.')
    if any(np.array(refImg.shape)-np.array(testImg.shape)):
        raise('Both images should have matching dims')
    refImg = img_norm(refImg.copy())
    testImg = img_norm(testImg.copy())
    refRGB = np.moveaxis(np.tile(refImg,(3,1,1)), 0, -1)
    testRGB = np.moveaxis(np.tile(testImg,(3,1,1)), 0, -1)
    blended = imblend(refImg, testImg)
    return np.array([refRGB, testRGB, blended])

def img_norm(img):
    return (img - np.amin(img)) / (np.amax(img) - np.amin(img))

def imblend(refImg, testImg):
    if (len(refImg.shape) != 2) or (len(testImg.shape) != 2):
        raise('Both images should have 2 dims.')
    if any(np.array(refImg.shape)-np.array(testImg.shape)):
        raise('Both images should have matching dims')
    Ly,Lx = refImg.shape
    blended = np.zeros((Ly,Lx,3))
    blended[:,:,0] = refImg
    blended[:,:,2] = testImg
    blended[:,:,1] = refImg
    return blended


h5Dir = 'D:/TPM/JK/h5/'
s2pDir = 'D:/TPM/JK/s2p/'
# mice =          [25,    27,   30,   36,     37,     38,     39,     41,     52,     53,     54,     56]
# refSessions =   [4,     3,    3,    1,      7,      2,      1,      3,      3,      3,      3,      3]
# expSessions =   [19,    10,   21,   17,     0,      0,      23,     0,      21,     0,      0,      0]
# zoom =          [2,     2,    2,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7,    1.7]
# freq =          [7.7,   7.7,  7.7,  7.7,    6.1,    6.1,    6.1,    6.1,    7.7,    7.7,    7.7,    7.7]

mice =          [25,    27,   30,   36,     39,     52]
refSessions =   [4,     3,    3,    1,      1,      3]
expSessions =   [19,    10,   21,   17,     23,     21]
zoom =          [2,     2,    2,    1.7,    1.7,    1.7]
freq =          [7.7,   7.7,  7.7,  7.7,    6.1,    7.7]

In [7]:
reg_result.keys()

dict_keys(['mouse', 'plane', 'edge_buffer', 'selected_session_i', 'selected_session_num', 'zdrift_list', 'registration_method', 'bilinear_result', 'affine_result', 'suite2p_result', 'old_result'])

In [21]:
reg_result['edge_buffer']

{'leftBuffer': 30, 'rightBuffer': 30, 'bottomBuffer': 10, 'topBuffer': 50}

In [11]:
reg_result_ops['tforms']

[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[ 1.00500151e+00, -4.07163571e-03, -1.19650678e-05,
         -5.03290797e+00],
        [ 9.26231526e-03,  1.00325726e+00, -3.70838721e-06,
         -2.63278773e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 1.00398015e+00, -1.16281364e-02, -1.16459484e-05,
          4.34427994e+00],
        [ 1.45844042e-02,  9.97511188e-01, -1.04486298e-06,
          2.92521511e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 1.00218259e+00, -1.72072899e-04, -8.43841476e-06,
          5.28684357e+00],
        [-7.00325158e-04,  9.99050499e-01, -6.38300285e-07,
          2.72447809e+00],
        [ 0.00000000

In [15]:
len(reg_result_ops['tforms'])

13

In [16]:
master_map_array.shape[0]

2275

In [19]:
master_map_merge = np.sum(master_map,axis=0)
master_map_merge.shape

(327, 627)

In [2]:
mi = 0
pn = 2

mouse = mice[mi]
planeDir = f'{h5Dir}{mouse:03}/plane_{pn}/'
load_fn = f'{planeDir}JK{mouse:03}_plane{pn}_roi_collection.npy'

# result = {'master_map_list': master_map_list,
# 'session_map_list': session_map_list,
# 'viable_cell_index_list': viable_cell_index_list,
# 'new_master_map_list': new_master_map_list,
# 'new_map_list': new_map_list,
# 'matching_master_roi_index_list': matching_master_roi_index_list,
#         }
        
result = np.load(load_fn, allow_pickle=True).item()

registration_fn = f'{planeDir}JK{mouse:03}_plane{pn}_session_to_session_registration.npy'
reg_result = np.load(registration_fn, allow_pickle=True).item()
reg_meth = reg_result['registration_method']
if reg_meth == 'old':
    reg_result_ops = reg_result['old_result']
elif reg_meth == 'suite2p':
    reg_result = fix_reg_collection(reg_result)
    reg_result_ops = reg_result['suite2p_result']
elif reg_meth == 'affine':
    reg_result_ops = reg_result['affine_result']
elif reg_meth == 'bilinear':
    reg_result_ops = reg_result['bilinear_result']
else:
    raise('Registration method mismatch.')
reg_img = reg_result_ops['reg_image']
n_session = len(reg_result['selected_session_num'])
master_map = result['master_map_list'][-1]

master_map_merge = np.sum(master_map,axis=0)

master_map_array = np.tile(master_map_merge,(n_session,1,1))
if (reg_meth == 'old') or (reg_meth == 'suite2p'):
    rigid_y1 = reg_result_ops['roff1'][0][0][si]
    rigid_x1 = reg_result_ops['roff1'][0][1][si]
    nonrigid_y1 = reg_result_ops['nroff1'][0][0][si]
    nonrigid_x1 = reg_result_ops['nroff1'][0][1][si]
    
    rigid_y2 = reg_result_ops['roff2'][0][0][si]
    rigid_x2 = reg_result_ops['roff2'][0][1][si]
    nonrigid_y2 = reg_result_ops['nroff2'][0][0][si]
    nonrigid_x2 = reg_result_ops['nroff2'][0][1][si]

    block_size1 = reg_result_ops['block_size1']
    block_size2 = reg_result_ops['block_size2']

    inverse_master_map = twostep_register_reverse(master_map_array, rigid_y1, rigid_x1, nonrigid_y1, nonrigid_x1, block_size1, 
                    rigid_y2, rigid_x2, nonrigid_y2, nonrigid_x2, block_size2)
else:
    inverse_master_map = stack_reg_reverse(master_map_array, reg_result_ops['tforms'], reg_meth)
inverse_master_map_ogdim = np.zeros()
reg_result['edge_buffer']
mean_image_list = []
for si in range(n_session):
    snum = reg_result['selected_session_num'][si]
    planeDir = f'{h5Dir}{mouse:03}/plane_{pn}/'
    ops = np.load(f'{planeDir}{snum:03}/plane0/ops.npy', allow_pickle=True).item()
    mean_image_list.append(ops['meanImg'])

viewer = napari.Viewer()
viewer.add_image(np.array(mean_image_list))
viewer.add_image(inverse_master_map)
    

FileNotFoundError: [Errno 2] No such file or directory: 'D:/TPM/JK/h5/025/plane_2/JK025_plane2_roi_collection.npy'